In [31]:
using CSV, DataFrames

In [32]:
Input_df = CSV.read("data/201215_datafile_all.csv", DataFrame);

In [33]:
describe(Input_df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,electkwh,0.353611,0.15532,0.350303,1.82909,0,Float64
2,heatingkwh,1.35625,0.0,0.027696,7.5231,0,Float64
3,hotwaterkwh,0.344568,0.0,0.0,5.50031,0,Float64
4,PV_generation,1.89333,0.0,0.007,10.0,0,Float64
5,Temperature,9.77827,-20.413,10.934,30.197,0,Float64
6,month,6.52603,1,7.0,12,0,Int64
7,day,15.7205,1,16.0,31,0,Int64
8,hour,11.5,0,11.5,23,0,Int64


In [34]:
# add new column with number of days
Input_df[!, :nday] = 1:nrow(Input_df)

1:8760

## Add periodical time representation using cos/sin

In [35]:
# add columns with cos and sin values for periodical time values day + month
Input_df[!, :hour_cos] = cos.(Input_df[!,:hour] ./ maximum(Input_df[!,:hour]) .* 2*pi);
Input_df[!, :hour_sin] = sin.(Input_df[!,:hour] ./ maximum(Input_df[!,:hour]) .* 2*pi);

# do not use month/day but number of day
#Input_df[!, :day_cos] = cos.(Input_df[!,:day] ./ maximum(Input_df[!,:day]) .* 2*pi);
#Input_df[!, :day_sin] = sin.(Input_df[!,:day] ./ maximum(Input_df[!,:day]) .* 2*pi);

Input_df[!, :month_cos] = cos.(Input_df[!,:month] ./ maximum(Input_df[!,:month]) .* 2*pi);
Input_df[!, :month_sin] = sin.(Input_df[!,:month] ./ maximum(Input_df[!,:month]) .* 2*pi);

#Input_df[!, :nday_cos] = cos.(Input_df[!,:nday] ./ maximum(Input_df[!,:nday]) .* 2*pi);
#Input_df[!, :nday_sin] = sin.(Input_df[!,:nday] ./ maximum(Input_df[!,:nday]) .* 2*pi);

In [45]:
describe(Input_df), size(Input_df)

(13×7 DataFrame
 Row │ variable       mean            min         median          max          ⋯
     │ Symbol         Float64         Real        Float64         Real         ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ electkwh          0.353611       0.15532      0.350303        1.82909   ⋯
   2 │ heatingkwh        1.35625        0.0          0.027696        7.5231
   3 │ hotwaterkwh       0.344568       0.0          0.0             5.50031
   4 │ PV_generation     1.89333        0.0          0.007          10.0
   5 │ Temperature       9.77827      -20.413       10.934          30.197     ⋯
   6 │ month             6.52603        1            7.0            12
   7 │ day              15.7205         1           16.0            31
   8 │ hour             11.5            0           11.5            23
   9 │ nday           4380.5            1         4380.5          8760         ⋯
  10 │ hour_cos          0.0416667     -0.990686     0.0676

## Extract training, testing + evalution data set for summer, winter, both, all

In [44]:
# summer June - August
Input_data_summer = filter(row -> row.month >= 6 && row.month  <= 8, Input_df)
Input_data_summer[!, "summer"] .= 1
Input_data_summer[!, "winter"] .= 0
describe(Input_data_summer), size(Input_data_summer)

(15×7 DataFrame
 Row │ variable       mean            min          median          max         ⋯
     │ Symbol         Float64         Real         Float64         Real        ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ electkwh          0.308262        0.15532      0.310149        1.18675  ⋯
   2 │ heatingkwh        0.00587495      0.0          0.0             1.23951
   3 │ hotwaterkwh       0.257596        0.0          0.0             5.50031
   4 │ PV_generation     2.26376         0.0          0.603           9.154
   5 │ Temperature      20.596           7.075       20.7915         29.626    ⋯
   6 │ month             7.01087         6            7.0             8
   7 │ day              15.837           1           16.0            31
   8 │ hour             11.5             0           11.5            23
   9 │ nday           4728.5          3625         4728.5          5832        ⋯
  10 │ hour_cos          0.0416667      -0.990686 

In [38]:
# summer split
summer_training = filter(row -> row.day <= 15, Input_data_summer)
summer_evaluation = filter(row -> row.day > 15 && row.day <= 20, Input_data_summer)
summer_testing = filter(row -> row.day > 20, Input_data_summer)

# write data files
CSV.write("data/summer_training.csv", summer_training);
CSV.write("data/summer_testing.csv", summer_testing);
CSV.write("data/summer_evaluation.csv", summer_evaluation);

In [43]:
# winter 
Input_data_winter = filter(row -> row.month >= 12 || row.month  <= 2, Input_df)
Input_data_winter[!, "summer"] .= 0
Input_data_winter[!, "winter"] .= 1
describe(Input_data_winter), size(Input_data_winter)

(15×7 DataFrame
 Row │ variable       mean            min            median          max       ⋯
     │ Symbol         Float64         Real           Float64         Real      ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ electkwh          0.40455        0.200706        0.40841         1.5557 ⋯
   2 │ heatingkwh        3.50821        0.01753         3.67684         7.5231
   3 │ hotwaterkwh       0.396046       0.0             0.0             5.5003
   4 │ PV_generation     1.32633        0.0             0.0            10.0
   5 │ Temperature      -2.88367      -20.413          -2.0965         14.845  ⋯
   6 │ month             5.1            1               2.0            12
   7 │ day              15.5333         1              15.5            31
   8 │ hour             11.5            0              11.5            23
   9 │ nday           3353.83           1            1080.5          8760      ⋯
  10 │ hour_cos          0.0416667     -0.

In [40]:
# winter split
winter_training = filter(row -> row.day <= 15, Input_data_winter)
winter_evaluation = filter(row -> row.day > 15 && row.day <= 20, Input_data_winter)
winter_testing = filter(row -> row.day > 20, Input_data_winter)

# write data files
CSV.write("data/winter_training.csv", winter_training);
CSV.write("data/winter_testing.csv", winter_testing);
CSV.write("data/winter_evaluation.csv", winter_evaluation);

In [41]:
# whole year split
all_training = filter(row -> row.day <= 15, Input_df)
all_evaluation = filter(row -> row.day > 15 && row.day <= 20, Input_df)
all_testing = filter(row -> row.day > 20, Input_df)

# write data files
CSV.write("data/all_training.csv", all_training);
CSV.write("data/all_testing.csv", all_testing);
CSV.write("data/all_evaluation.csv", all_evaluation);

In [46]:
describe(vcat(Input_data_winter, Input_data_summer) ), size(vcat(Input_data_winter, Input_data_summer) )

(15×7 DataFrame
 Row │ variable       mean            min         median          max          ⋯
     │ Symbol         Float64         Real        Float64         Real         ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ electkwh          0.355877       0.15532      0.326538        1.55576   ⋯
   2 │ heatingkwh        1.7378         0.0          0.0191135       7.5231
   3 │ hotwaterkwh       0.326061       0.0          0.0             5.50031
   4 │ PV_generation     1.8002         0.0          0.0045         10.0
   5 │ Temperature       8.98519      -20.413       12.6065         29.626     ⋯
   6 │ month             6.06593        1            7.0            12
   7 │ day              15.6868         1           16.0            31
   8 │ hour             11.5            0           11.5            23
   9 │ nday           4048.72           1         4392.5          8760         ⋯
  10 │ hour_cos          0.0416667     -0.990686     0.0676

In [47]:
# both seasons split
both_training = filter(row -> row.day <= 15, vcat(Input_data_winter, Input_data_summer) )
both_evaluation = filter(row -> row.day > 15 && row.day <= 20, vcat(Input_data_winter, Input_data_summer))
both_testing = filter(row -> row.day > 20, vcat(Input_data_winter, Input_data_summer))

# write data files
CSV.write("data/both_training.csv", both_training);
CSV.write("data/both_testing.csv", both_testing);
CSV.write("data/both_evaluation.csv", both_evaluation);